<a href="https://colab.research.google.com/github/Rams901/hands-on-llms/blob/main/llama2_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install accelerate bitsandbytes xformers adjustText transformers

In [2]:
from getpass import getpass
hf_token = getpass()

··········


In [3]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation types
)

# Llama 2 Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id, token = hf_token)

# Llama 2 Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
    token = hf_token,
  )

model.eval()

# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

prompt = "Could you explain to me how 4-bit quantization works as if I am 5?"
res = generator(prompt)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [6]:
res[0]['generated_text']

"Could you explain to me how 4-bit quantization works as if I am 5?\n everybody knows that 4 bits is not enough for most things, but it's a good way to understand the basics of digital signal processing.\nSo, imagine you have a big box full of different colored jelly beans. Each jelly bean represents a different value, like a number from 0 to 255. Now, let's say you want to give some of these jelly beans to your friends as gifts. But instead of giving them the whole jelly bean, you only give them a small piece of it.\nFor example, let's say you want to give your friend 3 jelly beans that are all different colors. You could take one of the jelly beans and break it into 3 smaller pieces, each representing a different color. This would be like 4-bit quantization because you are taking one big value (the whole jelly bean) and breaking it down into 3 smaller values (each piece of the jelly bean).\nBut here's the thing: even though you are giving your friend 3 different colored jelly beans, 